This notebook preprocess ensemble GEFS according to input used in Price and Rasp (2022)

Input data is 5 ensemble apcp, 5 ensemble pwat, cape, cin, and t2m

In [ ]:
!pip install netCDF4
import xarray as xr
from netCDF4 import Dataset
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
# import cartopy.crs as ccrs  # for plotting map
# import cartopy
import matplotlib as mpl
import requests
a = Dataset("finalprc.nc")
b = a.variables['lat'][:]
print(b)



[40.9517 39.047  37.1422]


In [ ]:
import xarray as xr
from datetime import datetime, timedelta

# Load your xarray dataset
rain = xr.open_dataset('finalhgt850.nc')

# Convert numpy.datetime64 to '2000-01-01T06:00' format
def convert_datetime64_to_format(dt64):
    target_date = (dt64 - np.datetime64('1800-01-01')) / np.timedelta64(1, 'h')
    return (datetime(1800, 1, 1) + timedelta(hours=target_date)).strftime('%Y-%m-%dT%H:%M')

# Convert the time coordinates to the desired format
new_time_coords = [convert_datetime64_to_format(dt64) for dt64 in rain.time.values]

# Replace the existing time coordinate with the new formatted time coordinates
rain = rain.assign_coords(time=new_time_coords)

print(rain)
a = rain.squeeze('level')
print(a)

FileNotFoundError: ignored

In [ ]:
hgt850 = xr.open_dataset("finalhgt850.nc")
hgt500 = xr.open_dataset("finalhgt500.nc")
rhum = xr.open_dataset("finalrhum.nc")
shum = xr.open_dataset("finalshum.nc")
temp = xr.open_dataset("finaltemp.nc")
slp = xr.open_dataset("finalmslp.nc")

hgt500 = hgt500.squeeze('level')
hgt850 = hgt850.squeeze("level")
shum = shum.squeeze("level")
print(shum)
final = [hgt850, hgt500, rhum, shum, temp, slp]
# for i in final:
#     # Convert the time coordinates to the desired format
#     new_time_coords = [convert_datetime64_to_format(dt64) for dt64 in i.time.values]

# # Replace the existing time coordinate with the new formatted time coordinates
#     i = rain.assign_coords(time=new_time_coords)
# print(hgt850)


def trainvaltest(dataset):
    '''
    Return train, val, test dataset
    '''
    train = dataset.sel(time=slice('1980-01-01T06', '2005-01-01T06'))
    val = dataset.sel(time=slice('2006-01-01T12', '2008-12-31T12'))
    test = dataset.sel(time=slice('2009-01-01T18', '2010-12-31T18'))
    return train, val, test

t,v,te = trainvaltest(rain)

<xarray.Dataset>
Dimensions:  (time: 45292, lon: 3, lat: 2)
Coordinates:
  * time     (time) datetime64[ns] 1980-01-01 ... 2010-12-31T18:00:00
  * lon      (lon) float32 280.0 282.5 285.0
  * lat      (lat) float32 40.0 37.5
    level    float32 850.0
Data variables:
    shum     (time, lat, lon) float32 ...
Attributes:
    CDI:            Climate Data Interface version 1.9.10 (https://mpimet.mpg...
    Conventions:    COARDS
    title:          4x daily NMC reanalysis (1980)
    description:    Data is from NMC initialized reanalysis\n(4x/day).  It co...
    platform:       Model
    dataset_title:  NCEP-NCAR Reanalysis 1
    References:     http://www.psl.noaa.gov/data/gridded/data.ncep.reanalysis...
    CDO:            Climate Data Operators version 1.9.10 (https://mpimet.mpg...
    history:        07-Aug-2023 18:45:15 Subset NOAA/PSL  


In [ ]:
# print(len(set(cin_c00.time.values)))
# print(len(cin_c00.time.values))
# print(len(set(pwat_p01.time.values)))
# print(len(pwat_p01.time.values))

# have duplicated dates in pwat_p01, p02, p03, p04
# print(len(set(pwat_p01.time.values)))
# print(len(pwat_p01.time.values))

# to find the indices of duplicated value
# pwat_duplicates = list(pwat_p01.get_index("time").duplicated())
# indices = [i for i, x in enumerate(pwat_duplicates) if x == True]
# indices

# check if duplicates are due to error in date naming
# pwat_p01.isel(time=6200).pwat.values
# pwat_p01.isel(time=9116).pwat.values
# the duplicates are exactly the same, so it is not a date naming error

In [ ]:
hgt850_train, hgt850_val, hgt850_test = trainvaltest(hgt850)
hgt500_train, hgt500_val, hgt500_test = trainvaltest(hgt500)
rhum_train, rhum_val, rhum_test = trainvaltest(rhum)
shum_train, shum_val, shum_test = trainvaltest(shum)
slp_train, slp_val, slp_test = trainvaltest(slp)
temp_train, temp_val, temp_test = trainvaltest(temp)

In [ ]:
def resampling(partial_dataset, var):
    partial_dataset = partial_dataset.resample(time='6H').asfreq()
    if var == 'hgt':
        partial_dataset.hgt.values = partial_dataset.hgt.interpolate_na(dim='time')
        return partial_dataset
    if var == 'slp':
        partial_dataset.slp.values = partial_dataset.slp.interpolate_na(dim='time')
        return partial_dataset
    if var == 'rhum':
        partial_dataset.rhum.values = partial_dataset.rhum.interpolate_na(dim='time')
        return partial_dataset
    if var == 'shum':
        partial_dataset.shum.values = partial_dataset.shum.interpolate_na(dim='time')
        return partial_dataset
    if var == 'tmp':
        partial_dataset.tmp.values = partial_dataset.tmp.interpolate_na(dim='time')
        return partial_dataset
    else:
        print('Error: var not found in function, please define')

In [ ]:
hgt850_train = resampling(hgt850_train, 'hgt')
hgt500_train = resampling(hgt500_train, 'hgt')
rhum_train = resampling(rhum_train, 'rhum')
shum_train = resampling(shum_train, 'shum')
slp_train = resampling(slp_train, 'slp')
temp_train = resampling(temp_train, 'tmp')

In [ ]:
# import scipy as sp
# # a=cape_train_c00.cape.values
# # a = pwat_train_c00.pwat.values

# a=t2m_train_c00.t2m.values

# p=sp.stats.mstats.normaltest(a, axis=0).pvalue
# if p.all()<0.01:
#    print ('distribution is not normal')
# p=sp.stats.mstats.normaltest(np.log(a), axis=0).pvalue
# if p.all()<0.01:
#    print ('distribution is not log-normal')

In [ ]:
# ALL MISSING DATES LIE IN TRAINING DATASET

# print(timecheck(apcp_train_p01, apcp_val_p01, apcp_test_p01))
# print(timecheck(apcp_train_p02, apcp_val_p02, apcp_test_p02))
# print(timecheck(apcp_train_p03, apcp_val_p03, apcp_test_p03))
# print(timecheck(apcp_train_p04, apcp_val_p04, apcp_test_p04))
# print(timecheck(apcp_train_c00, apcp_val_c00, apcp_test_c00))

# print(timecheck(pwat_train_p01, pwat_val_p01, pwat_test_p01))
# print(timecheck(pwat_train_p02, pwat_val_p02, pwat_test_p02))
# print(timecheck(pwat_train_p03, pwat_val_p03, pwat_test_p03))
# print(timecheck(pwat_train_p04, pwat_val_p04, pwat_test_p04))
# print(timecheck(pwat_train_c00, pwat_val_c00, pwat_test_c00))

# print(timecheck(cape_train_c00, cape_val_c00, cape_test_c00))
# print(timecheck(t2m_train_c00, t2m_val_c00, t2m_test_c00))
# print(timecheck(cin_train_c00, cin_val_c00, cin_test_c00))

In [ ]:
def transform_train(trainds, var):
    if var == 'slp': # tp
        scaler_train = MinMaxScaler()
        X_one_col = trainds.slp.values.reshape([trainds.slp.values.shape[0]*trainds.slp.values.shape[1]*trainds.slp.values.shape[2], 1])
        X_one_col = np.log10(X_one_col+1) # 10**X_one_col - 1 to scale back
        X_one_col_res = scaler_train.fit_transform(X_one_col) # scaler_train.inverse_transform(X_one_col_res) to scale back, or use 10**scaler_train.inverse_transform(X_one_col_res) -1 only
        trainds.slp.values = X_one_col_res.reshape(trainds.slp.values.shape)
        return scaler_train, trainds

    if var == 'rhum': # tp
        scaler_train = MinMaxScaler()
        X_one_col = trainds.rhum.values.reshape([trainds.rhum.values.shape[0]*trainds.rhum.values.shape[1]*trainds.rhum.values.shape[2], 1])
        X_one_col = np.log10(X_one_col+1) # 10**X_one_col - 1 to scale back
        X_one_col_res = scaler_train.fit_transform(X_one_col) # scaler_train.inverse_transform(X_one_col_res) to scale back, or use 10**scaler_train.inverse_transform(X_one_col_res) -1 only
        trainds.rhum.values = X_one_col_res.reshape(trainds.rhum.values.shape)
        return scaler_train, trainds

    if var == 'shum':
        scaler_train = MinMaxScaler()
        X_one_col = trainds.shum.values.reshape([trainds.shum.values.shape[0]*trainds.shum.values.shape[1]*trainds.shum.values.shape[2], 1])
        # X_one_col = np.log10(X_one_col+1) # 10**X_one_col - 1 to scale back
        X_one_col_res = scaler_train.fit_transform(X_one_col) # scaler_train.inverse_transform(X_one_col_res) to scale back, or use 10**scaler_train.inverse_transform(X_one_col_res) -1 only
        trainds.shum.values = X_one_col_res.reshape(trainds.shum.values.shape)
        return scaler_train, trainds

    if var == 'tmp':
        scaler_train = MinMaxScaler()
        X_one_col = trainds.tmp.values.reshape([trainds.tmp.values.shape[0]*trainds.tmp.values.shape[1]*trainds.tmp.values.shape[2], 1])
        # X_one_col = np.log10(X_one_col+1) # 10**X_one_col - 1 to scale back
        X_one_col_res = scaler_train.fit_transform(X_one_col) # scaler_train.inverse_transform(X_one_col_res) to scale back, or use 10**scaler_train.inverse_transform(X_one_col_res) -1 only
        trainds.tmp.values = X_one_col_res.reshape(trainds.tmp.values.shape)
        return scaler_train, trainds

    if var == 'hgt':
        scaler_train = MinMaxScaler()
        X_one_col = trainds.hgt.values.reshape([trainds.hgt.values.shape[0]*trainds.hgt.values.shape[1]*trainds.hgt.values.shape[2], 1])
        # X_one_col = np.log10(X_one_col+1) # 10**X_one_col - 1 to scale back
        X_one_col_res = scaler_train.fit_transform(X_one_col) # scaler_train.inverse_transform(X_one_col_res) to scale back, or use 10**scaler_train.inverse_transform(X_one_col_res) -1 only
        trainds.hgt.values = X_one_col_res.reshape(trainds.hgt.values.shape)
        return scaler_train, trainds

    if var == 't2m':
        scaler_train = MinMaxScaler()
        X_one_col = trainds.t2m.values.reshape([trainds.t2m.values.shape[0]*trainds.t2m.values.shape[1]*trainds.t2m.values.shape[2], 1])
        # X_one_col = np.log10(X_one_col+1) # 10**X_one_col - 1 to scale back
        X_one_col_res = scaler_train.fit_transform(X_one_col) # scaler_train.inverse_transform(X_one_col_res) to scale back, or use 10**scaler_train.inverse_transform(X_one_col_res) -1 only
        trainds.t2m.values = X_one_col_res.reshape(trainds.t2m.values.shape)
        return scaler_train, trainds

    else:
        print('Error: var not found in function, please define')

transforming train, val and test datasets

In [ ]:
scaler_train_hgt850,hgt850_train = transform_train(hgt850_train,'hgt')
scaler_train_hgt500,hgt500_train = transform_train(hgt500_train, 'hgt')
scaler_train_slp,slp_train = transform_train(slp_train,'slp')
scaler_train_rhum,rhum_train= transform_train(rhum_train, 'rhum')
scaler_train_shum,shum_train = transform_train(shum_train, 'shum')
scaler_train_temp,temp_train = transform_train(temp_train, 'tmp')

In [ ]:
def transform_val_test(val_test, scaler_train, is_prec=True):
    '''
    Input (example): ds_val_apcp.tp, scaler_train, True/False
    Output: Transformed validation/test XR data
    If is_prec set to True, variable is precipitation
    '''
    if is_prec:
        X_one_col = val_test.values.reshape([val_test.values.shape[0]*val_test.values.shape[1]*val_test.values.shape[2], 1])
        X_one_col = np.log10(X_one_col+1)
        X_one_col_res = scaler_train.transform(X_one_col)
        val_test.values = X_one_col_res.reshape(val_test.values.shape)
        return val_test

    else:
        X_one_col = val_test.values.reshape([val_test.values.shape[0]*val_test.values.shape[1]*val_test.values.shape[2], 1])
        X_one_col_res = scaler_train.transform(X_one_col)
        val_test.values = X_one_col_res.reshape(val_test.values.shape)
        return val_test


In [ ]:
slp_val = transform_val_test(slp_val.slp, scaler_train_slp,is_prec = False)
rhum_val = transform_val_test(rhum_val.rhum, scaler_train_rhum,is_prec = False)
shum_val = transform_val_test(shum_val.shum, scaler_train_shum,is_prec = False)
temp_val = transform_val_test(temp_val.tmp, scaler_train_temp,is_prec = False)
hgt850_val = transform_val_test(hgt850_val.hgt, scaler_train_hgt850,is_prec = False)
hgt500_val = transform_val_test(hgt500_val.hgt, scaler_train_hgt500,is_prec = False)


slp_test = transform_val_test(slp_test.slp, scaler_train_slp, is_prec =  False)
rhum_test = transform_val_test(rhum_test.rhum, scaler_train_rhum, is_prec =  False)
shum_test = transform_val_test(shum_test.shum, scaler_train_shum, is_prec =  False)
temp_test = transform_val_test(temp_test.tmp, scaler_train_temp, is_prec =  False)
hgt850_test = transform_val_test(hgt850_test.hgt, scaler_train_hgt850, is_prec =  False)
hgt500_test = transform_val_test(hgt500_test.hgt, scaler_train_hgt500, is_prec =  False)

In [ ]:
print(slp_train)
shum_train =shum_train.values

converting to .npy files

In [ ]:

slp_train_data  = slp_train['slp']
slp_train_final = slp_train_data.values
print(slp_train_final)
print(slp_train_final.shape)



hgt850_train_data = hgt850_train['hgt']
hgt850_train_final = hgt850_train_data.values

hgt500_train_data = hgt500_train['hgt']
hgt500_train_final = hgt500_train_data.values

temp_train_data = temp_train['tmp']
temp_train_final = temp_train_data.values

rhum_train_data = rhum_train['rhum']
rhum_train_final = rhum_train_data.values

shum_train_data = shum_train['shum']
shum_train_final = shum_train_data.values


In [ ]:
lst_train_ensemble = [hgt850_train_final, hgt500_train_final, slp_train_final,temp_train_final, rhum_train_final, shum_train_final]
#print(lst_train_ensemble)
X_train_ensemble = np.stack((lst_train_ensemble), axis = -1)
print(X_train_ensemble)





combining variables into 1 4D array

In [ ]:
lst_train_ensemble = [hgt850_train_final, hgt500_train_final, slp_train_final, rhum_train_final, shum_train_final]

X_train_ensemble = np.stack((lst_train_ensemble), axis = -1) # stacking 13 variables into 1 single 4D array

# lst_val_ensemble = [hgt850_val, hgt500_val, slp_train,temp_val, rhum_val, shum_val]

# X_val_ensemble = np.stack((lst_val_ensemble), axis = -1) # stacking 13 variables into 1 single 4D array

# lst_test_ensemble = [hgt850_test, hgt500_test, slp_test,temp_test, rhum_test, shum_test]

# X_test_ensemble = np.stack((lst_test_ensemble), axis = -1) # stacking 13 variables into 1 single 4D array

print(X_train_ensemble.shape)

In [ ]:
from google.colab import files
 # please change this to your own path
np.save('X_train_ensemble.npy', X_train_ensemble)

 np.save('X_val_ensemble.npy', X_val_ensemble)
 np.save('X_test_ensemble.npy', X_test_ensemble)

 files.download('X_train_ensemble.npy')
 files.download('X_val_ensemble.npy')
 files.download('X_test_ensemble.npy')